In [2]:
#!/usr/bin/env python

###############
# Main method #
###############

#TODO: What is this function supposed to do? 

__author__ = "Pär-Love Palm, Felix Steimle, Jakob Wadman, Veit Wörner"
__copyright__ = "Copyright 2019, Chalmers University of Technology"
__credits__ = ["Pär-Love Palm", "Felix Steimle", "Jakob Wadman", "Veit Wörner"]
__license__ = "GPL"
__version__ = "0.9b"
__maintainer__ = "Veit Wörner"
__email__ = "veit@student.chalmers.se"
__status__ = "Production"

from numpy import array
from multiprocessing import Process, Queue

from visualize_combination_code import *
from Simulate_combination_code import *
from utility_functions import *

truck_translation = array([np.float(18),np.float(5)])
truck_rotation = 0
first_trailer_rotation = 20
second_trailer_rotation = 20
destination_translation = array([4.5,5])
destination_rotation = 180
number_trailers = 2
step_size = 1e-2

rotation_center_truck = array([2.5,1])
rotation_center_first_trailer = array([1.5,1])
rotation_center_second_trailer = array([2.5,1])

truck_shape = array([7,2])
first_trailer_shape = array([2,2])
second_trailer_shape = array([9,2])

simulate_combination = Simulate_combination(truck_translation,\
                                            truck_rotation,\
                                            first_trailer_rotation,\
                                            second_trailer_rotation,\
                                            destination_translation,\
                                            destination_rotation,\
                                            number_trailers,\
                                            step_size)    
        
def visualize(visualisation_queue): # ,overlap_queue,bound_queue):
    plotting_number = 1
    plotting_interval = 1e2
    
    while True:
        if np.mod(plotting_number,plotting_interval) == 0:
            print('') # Deleting this line causes the system to plot in the shell instead of in Jupyter
            # print(overlap_queue.get()) 
            # print(bound_queue.get())
            plotting_number = 1
            visualisation_element = visualisation_queue.get()
            if (visualisation_element == 'DONE'):
                break
            visualize_combination(visualisation_element)
        else:
            plotting_number += 1
            # overlap_queue.get()
            # bound_queue.get()
            visualisation_queue.get()
                
def main():
    visualisation_queue = Queue()
    # overlap_queue = Queue()
    # bound_queue = Queue()
    visualisation_process = Process(target=visualize, args=((visualisation_queue),)) # (overlap_queue),(bound_queue),))
    visualisation_process.daemon = True
    visualisation_process.start()
    
    for step_number in range(np.int(1e4)):
        velocity = -0.1+np.sin(step_number/1e3*np.pi)
        steering_percentage = np.sin(step_number/1e2*np.pi)
            
        truck_translation,\
        truck_rotation,\
        first_trailer_rotation,\
        second_trailer_rotation = simulate_combination.run(velocity,steering_percentage)
#        overlap = destination_overlap([rotation_center_truck,rotation_center_first_trailer,rotation_center_second_trailer],\
#                                      [first_trailer_rotation,second_trailer_rotation],\
#                                      destination_translation,\
#                                      destination_rotation,\
#                                      trailer_shape_list=[first_trailer_shape,second_trailer_shape])
#        bounds = out_of_bounds([rotation_center_truck,rotation_center_first_trailer,rotation_center_second_trailer],\
#                               [truck_rotation,first_trailer_rotation,second_trailer_rotation],\
#                               [truck_shape,first_trailer_shape,second_trailer_shape])
        
        visualisation_queue.put([truck_translation, 
                                 truck_rotation,       
                                 first_trailer_rotation,      
                                 second_trailer_rotation,      
                                 steering_percentage,                 
                                 destination_translation,          
                                 destination_rotation,          
                                 number_trailers])
#        overlap_queue.put(overlap)
#        bound_queue.put()
        
    visualisation_queue.put('DONE')    
    visualisation_process.join()
    
if __name__=='__main__':
    main()    